# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4815, uptime 0:00:33
mini_beamline                    RUNNING   pid 4816, uptime 0:00:33
random_walk                      RUNNING   pid 4831, uptime 0:00:31
random_walk_horiz                RUNNING   pid 4832, uptime 0:00:31
random_walk_vert                 RUNNING   pid 4819, uptime 0:00:33
simple                           RUNNING   pid 4820, uptime 0:00:33
thermo_sim                       RUNNING   pid 4821, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4822, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 17:04:02
Persistent Unique Scan ID: '591c6cd3-2508-44a3-8dfb-7f81c98a867b'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 17:04:02.2 |     -1.000 |     102579 |
|         2 | 17:04:02.2 |     -0.500 |     104138 |
|         3 | 17:04:02.2 |      0.000 |     104864 |
|         4 | 17:04:02.3 |      0.500 |     104601 |
|         5 | 17:04:02.3 |      1.000 |     102489 |


+-----------+------------+------------+------------+
generator scan ['591c6cd3'] (scan num: 3)





('591c6cd3-2508-44a3-8dfb-7f81c98a867b',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/591c6cd3-2508-44a3-8dfb-7f81c98a867b-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 17:04:02
Persistent Unique Scan ID: 'fd9f4b07-9ba0-40f0-82b7-3355e2c69f8a'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 17:04:02.8 |          -1 | 1129209690 |


|         2 | 17:04:03.0 |          -0 | 1125718115 |


|         3 | 17:04:03.1 |           0 | 1117215606 |
|         4 | 17:04:03.2 |           0 | 1110791798 |


|         5 | 17:04:03.2 |           1 | 1105138286 |


+-----------+------------+-------------+------------+
generator scan ['fd9f4b07'] (scan num: 4)





('fd9f4b07-9ba0-40f0-82b7-3355e2c69f8a',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 17:04:03
Persistent Unique Scan ID: 'b526093f-4ea9-4fed-b233-191d6bdd9ec2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 17:04:03.8 | 1080754146 |


|         2 | 17:04:04.7 | 1055738634 |


|         3 | 17:04:05.7 | 1106707198 |


...

|        28 | 17:04:30.7 | 1128278959 |


|        29 | 17:04:31.7 | 1075935410 |


|        30 | 17:04:32.7 | 1057085141 |


+-----------+------------+------------+
generator count ['b526093f'] (scan num: 5)





('b526093f-4ea9-4fed-b233-191d6bdd9ec2',)